In [4]:
!pip3 install torch torchvision
!pip3 install transformers
!pip3 install scikit-learn
!pip3 install pandas
!pip3 install numpy

     |████████████████████████████████| 150.8 MB 297 kB/s  eta 0:00:01
     |████████████████████████████████| 1.7 MB 10.9 MB/s eta 0:00:01
  Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
     |████████████████████████████████| 1.6 MB 11.3 MB/s eta 0:00:01
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_10_9_x86_64.whl (21.2 MB)
  Using cached pillow-11.3.0-cp39-cp39-macosx_10_10_x86_64.whl (5.3 MB)
  Using cached markupsafe-3.0.3-cp39-cp39-macosx_10_9_x86_64.whl (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
You should consider upgrading via the '/Users/abby/Documents/NCCU/visualized/Dog-breed-data-visualization-/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 12.0 MB 12.1 MB/s eta 0:00:01
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_10_12_x86_64.whl (

In [8]:
import pandas as pd

# Set random seed for reproducibility
random_seed = 42

# sampling
# load csv to pandas dataframe
csv_file_path = "static/dogs/dogs.csv"
data = pd.read_csv(csv_file_path)

# random sample 28 data from each label
sampled_df = data.groupby('labels').apply(lambda x: x.sample(n=min(28, len(x)),random_state=random_seed)).reset_index(drop=True)

print(sampled_df)

                 filepaths  labels data set
0     train/Afghan/019.jpg  Afghan    train
1     train/Afghan/043.jpg  Afghan    train
2     train/Afghan/037.jpg  Afghan    train
3     train/Afghan/077.jpg  Afghan    train
4     train/Afghan/054.jpg  Afghan    train
...                    ...     ...      ...
1965   train/Yorkie/16.jpg  Yorkie    train
1966   train/Yorkie/40.jpg  Yorkie    train
1967   train/Yorkie/34.jpg  Yorkie    train
1968   train/Yorkie/10.jpg  Yorkie    train
1969    test/Yorkie/03.jpg  Yorkie     test

[1970 rows x 3 columns]


/var/folders/29/b83bvxsd419fp15jy0g1y_400000gn/T/ipykernel_74852/1388297045.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = data.groupby('labels').apply(lambda x: x.sample(n=min(28, len(x)),random_state=random_seed)).reset_index(drop=True)


In [ ]:
from tqdm import tqdm # **************a useful tools for counting the loop**************
from PIL import Image
import torch
from transformers import EfficientNetImageProcessor
from torchvision.models import efficientnet_b5
import os

# random seed (may unessary)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# set up device when you can use gpu by cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load pretrain efficientNet model
preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b5")

model = efficientnet_b5(pretrained=True)
model = model.to(device)
model.eval()

# calculate the feature vector by model
tmp_feature_vector = []

for p in tqdm(sampled_df["image_path"].to_list()):
    with torch.no_grad():
        # torch.cuda.empty_cache() # if out of memory
        image = Image.open("static/dogs/" + p)
        # image = Image.open("../van-gogh-paintings" + p)
        input_tensor = preprocessor(image, return_tensors="pt")["pixel_values"]
        input_tensor = input_tensor.to(device)
        features = model.features(input_tensor)
        feature_vector = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1))
        feature_vector = feature_vector.squeeze()

        # Move feature vector to CPU and convert to numpy array
        tmp_feature_vector.append(feature_vector.cpu().numpy())

# Add feature vectors as a new column in the dataframe
sampled_df["feature_vector"] = tmp_feature_vector


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe210943190>>
Traceback (most recent call last):
  File "/Users/abby/Documents/NCCU/visualized/Dog-breed-data-visualization-/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    

In [4]:
import numpy as np
from sklearn.manifold import TSNE

# use tsne for dim reduction (to 2d as visualization requirement)
X = np.array(tmp_feature_vector)
X_embedded = TSNE(n_components=2, random_state=random_seed).fit_transform(X)

print(X_embedded.shape, X_embedded)

(308, 2) [[  8.841568    -2.775674  ]
 [  1.6205475    0.9120784 ]
 [ -9.33711     -4.1577196 ]
 [ 18.145416    -1.569106  ]
 [  2.1939228   -8.109316  ]
 [  2.0785046    4.8222184 ]
 [ 17.976805    -1.5183275 ]
 [ -7.102731    -2.110755  ]
 [  7.725385     4.58669   ]
 [ -8.187394    -3.362206  ]
 [ -4.387518    -1.7696745 ]
 [-11.613907    -3.9353316 ]
 [ -7.293229    -9.541788  ]
 [ 12.406604    -1.9566987 ]
 [  0.41665268 -12.4359865 ]
 [  8.675603    -7.553959  ]
 [  4.226828    -6.6129947 ]
 [ -6.090462     2.4739232 ]
 [ 12.2517805    0.92434293]
 [ 11.828333    10.053828  ]
 [ 13.178735     1.9852179 ]
 [ -3.828156     1.083201  ]
 [  6.6644206    0.5359933 ]
 [ -3.702994    -3.1244204 ]
 [-11.703681    -3.263283  ]
 [  8.073968     4.5771294 ]
 [  8.439203     0.22820292]
 [-11.98715     -3.3576212 ]
 [ -1.7871562   -4.2257185 ]
 [ -2.317896    -5.208047  ]
 [ -2.7818394   -3.5912504 ]
 [ -4.0332327   -7.3742533 ]
 [ -4.299238    -4.426341  ]
 [ 10.252804    -3.6148868 ]
 [  7

In [5]:
# add 2d dim as x, y coordinate to sampled_df
sampled_df['x'] = X_embedded[:, 0]
sampled_df['y'] = X_embedded[:, 1]

# output file to csv as processed dataset
print(sampled_df)
sampled_df.to_csv('output_file.csv', index=False)


                                            image_path  \
0    /kaggle/input/van-gogh-paintings/Arles/The Bab...   
1    /kaggle/input/van-gogh-paintings/Arles/Three S...   
2    /kaggle/input/van-gogh-paintings/Arles/Ploughe...   
3    /kaggle/input/van-gogh-paintings/Arles/Portrai...   
4    /kaggle/input/van-gogh-paintings/Arles/Still L...   
..                                                 ...   
303  /kaggle/input/van-gogh-paintings/Works of the ...   
304  /kaggle/input/van-gogh-paintings/Works of the ...   
305  /kaggle/input/van-gogh-paintings/Works of the ...   
306  /kaggle/input/van-gogh-paintings/Works of the ...   
307  /kaggle/input/van-gogh-paintings/Works of the ...   

                      class_name  labels  \
0                          Arles       1   
1                          Arles       1   
2                          Arles       1   
3                          Arles       1   
4                          Arles       1   
..                           ...     ..